<img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Profissão: Cientista de Dados**
### **Módulo 13** | Exercício 1 | Regressão Múltipla

Aluno [Lays Félix](https://www.linkedin.com/in/laysfelixbusiness)<br>
Data: 29 de junho de 2024

---

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import numpy as np
import pandas as pd
import patsy

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
prev_renda = pd.read_csv('previsao_de_renda.csv')

In [3]:
prev_renda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
prev_renda.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
# Verificando a frequência das categorias em cada variável:

for coluna in prev_renda.columns[3:]:
    print(coluna, '\n', prev_renda[coluna].value_counts(), '\n')

sexo 
 F    10119
M     4881
Name: sexo, dtype: int64 

posse_de_veiculo 
 False    9140
True     5860
Name: posse_de_veiculo, dtype: int64 

posse_de_imovel 
 True     10143
False     4857
Name: posse_de_imovel, dtype: int64 

qtd_filhos 
 0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: qtd_filhos, dtype: int64 

tipo_renda 
 Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64 

educacao 
 Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64 

estado_civil 
 Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64 

tipo_residencia 
 Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comu

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.


Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.

Mantenha sempre a categoria mais frequente como casela de referência.

Avalie os parâmetros e veja se parecem fazer sentido prático.

In [6]:
y, X = patsy.dmatrices('''np.log(renda) ~ sexo + posse_de_veiculo 
                                               + posse_de_imovel 
                                               + C(qtd_filhos, Treatment(0))
                                               + C(tipo_renda, Treatment('Assalariado'))
                                               + C(educacao, Treatment('Secundário')) 
                                               + C(estado_civil, Treatment('Casado')) 
                                               + C(tipo_residencia, Treatment('Casa')) 
                                               + idade 
                                               + tempo_emprego 
                                               + C(qt_pessoas_residencia, Treatment(2))
                                    ''', data=prev_renda)

In [7]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        12:31:47   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0932      0.039    183.905      0.000       7.018       7.169
sexo[T.M]                                                       0.7878      0.015     53.664      0.000       0.759       0.817
posse_de_veiculo[T.True]                                        0.0436      0.014      3.079      0.002       0.016       0.071
posse_de_imovel[T.True]                                         0.0828      0.014      5.907      0.000       0.055       0.110
C(qtd_filhos, Treatment(0))[T.1]                               -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos, Treatment(0))[T.2]                               -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos, Treatment(0))[T.4]                                0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos, Treatment(0))[T.5]                                0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos, Treatment(0))[T.7]                                0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos, Treatment(0))[T.14]                              -0.4630      0.261     -1.772      0.076      -0.975       0.049
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2259      0.241      0.937      0.349      -0.247       0.699
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1547      0.015     10.359      0.000       0.125       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2891      0.243     -1.190      0.234      -0.765       0.187
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment('Secundário'))[T.Primário]                0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.897      0.370      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3176      0.112      2.831      0.005       0.098       0.537
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2556      0.110      2.319      0.020       0.040       0.472
C(estado_ci

Resposta: Não parece fazer sentido porque AIC está muito elevado (2.719e+04), tendo em vista que menores valores de AIC representam uma maior qualidade e simplicidade.

_______________________

2. Remova a variável menos significante e analise: 

Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.

Observe os parâmetros e veja se algum se alterou muito.

In [9]:
# Usando o indicador "p-value", exclui as variáveis com valor menor '0,05': posse_de_veiculo, 
# posse_de_imovel, estado_civil, idade e tempo_emprego

smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + C(qtd_filhos, Treatment(0))
                                + C(tipo_renda, Treatment('Assalariado'))
                                + C(educacao, Treatment('Secundário'))  
                                + C(qt_pessoas_residencia, Treatment(2))
                ''', 
        data=prev_renda).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     134.3
Date:                Sat, 29 Jun 2024   Prob (F-statistic):               0.00
Time:                        12:53:34   Log-Likelihood:                -18130.
No. Observations:               15000   AIC:                         3.630e+04
Df Residuals:                   14978   BIC:                         3.647e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.9630      0.014    574.346      0.000       7.936       7.990
sexo[T.M]                                                       0.6936      0.015     47.554      0.000       0.665       0.722
C(qtd_filhos, Treatment(0))[T.1]                               -0.0199      0.036     -0.553      0.580      -0.090       0.051
C(qtd_filhos, Treatment(0))[T.2]                               -0.1160      0.103     -1.122      0.262      -0.319       0.087
C(qtd_filhos, Treatment(0))[T.3]                                0.3213      0.348      0.924      0.356      -0.360       1.003
C(qtd_filhos, Treatment(0))[T.4]                                0.2144      0.477      0.450      0.653      -0.720       1.149
C(qtd_filhos, Treatment(0))[T.5]                                0.0527      0.456      0.116      0.908      -0.841       0.947
C(qtd_filhos, Treatment(0))[T.7]                               -0.1699      0.181     -0.936      0.349      -0.526       0.186
C(qtd_filhos, Treatment(0))[T.14]                              -0.3517      0.287     -1.226      0.220      -0.914       0.211
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.3966      0.271      1.465      0.143      -0.134       0.927
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.0689      0.017      4.136      0.000       0.036       0.102
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.1491      0.020     -7.640      0.000      -0.187      -0.111
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.2169      0.025      8.758      0.000       0.168       0.265
C(educacao, Treatment('Secundário'))[T.Primário]                0.0043      0.064      0.068      0.946      -0.121       0.129
C(educacao, Treatment('Secundário'))[T.Pós graduação]          -0.0425      0.160     -0.266      0.790      -0.356       0.271
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.0942      0.014      6.601      0.000       0.066       0.122
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.1353      0.035     -3.865      0.000      -0.204      -0.067
C(qt_pessoas_residencia, Treatment(2))[T.1.0]                  -0.0136      0.018     -0.748      0.455      -0.049       0.022
C(qt_pessoas_residencia, Treatment(2))[T.3.0]                  -0.0625      0.038     -1.640      0.101      -0.137       0.012
C(qt_pessoas_residencia, Treatment(2))[T.4.0]                   0.1355      0.106      1.283      0.199      -0.071       0.342
C(qt_pessoas_residencia, Treatment(2))[T.5.0]                  -0.4909      0.353     -1.391      0.164      -1.183       0.201
C(qt_pessoa

Resposta: O 2º modelo piorou comparando os valores do R² e R²-ajustado com os valores do 1º modelo.

______________________________________

3. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%.

Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [18]:
# exclui as variáveis: sexo, 

smf.ols(formula='''
                np.log(renda) ~ qtd_filhos == 1
                                + qtd_filhos == 4
                                + qtd_filhos == 5
                                + qtd_filhos == 7
                                + qtd_filhos == 14
                                + tipo_renda == 'Bolsista'
                                + educacao == 'Primário'
                                + educacao == 'Pós graduação'
                                + C(qt_pessoas_residencia, Treatment(2))                     
                ''', 
        data=prev_renda).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     8.353
Date:                Sat, 29 Jun 2024   Prob (F-statistic):           4.14e-17
Time:                        13:21:01   Log-Likelihood:                -19370.
No. Observations:               15000   AIC:                         3.877e+04
Df Residuals:                   14986   BIC:                         3.888e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          8.2091      0.010    813.202      0.000       8.189       8.229
qtd_filhos == 1[T.True]                            0.0033      0.037      0.090      0.928      -0.068       0.075
qtd_filhos == 4[T.True]                            0.4244      0.518      0.820      0.412      -0.590       1.439
qtd_filhos == 5[T.True]                            0.0499      0.495      0.101      0.920      -0.921       1.021
qtd_filhos == 7[T.True]                           -0.2929      0.197     -1.487      0.137      -0.679       0.093
qtd_filhos == 14[T.True]                          -0.1279      0.311     -0.411      0.681      -0.738       0.482
tipo_renda == 'Bolsista'[T.True]                   0.2448      0.294      0.833      0.405      -0.331       0.820
educacao == 'Primário'[T.True]                    -0.0587      0.069     -0.851      0.395      -0.194       0.077
educacao == 'Pós graduação'[T.True]               -0.1167      0.173     -0.674      0.500      -0.456       0.223
C(qt_pessoas_residencia, Treatment(2))[T.1.0]     -0.1053      0.020     -5.382      0.000      -0.144      -0.067
C(qt_pessoas_residencia, Treatment(2))[T.3.0]      0.0136      0.039      0.351      0.726      -0.062       0.089
C(qt_pessoas_residencia, Treatment(2))[T.4.0]      0.1906      0.026      7.244      0.000       0.139       0.242
C(qt_pessoas_residencia, Treatment(2))[T.5.0]     -0.0423      0.067     -0.635      0.526      -0.173       0.088
C(qt_pessoas_residencia, Treatment(2))[T.6.0]     -0.1583      0.495     -0.320      0.749      -1.129       0.812
C(qt_pessoas_residencia, Treatment(2))[T.7.0]      0.6325      0.796      0.795      0.427      -0.927       2.192
C(qt_pessoas_residencia, Treatment(2))[T.9.0]     -0.2929      0.197     -1.487      0.137      -0.679       0.093
C(qt_pessoas_residencia, Treatment(2))[T.15.0]    -0.1279      0.311     -0.411      0.681      -0.738       0.482
==============================================================================
Omnibus:                      309.501   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              350.043
Skew:                           0.316   Prob(JB):                     9.75e-77
Kurtosis:                       3.400   Cond. No.                     2.02e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.15e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Resposta: O 3º modelo também piorou porque os indicadores R² e R²-ajustado continuou com valores abaixo do 2º modelo comparado. Entretanto, o 1º modelo parece ser o melhor porque tem o coeficiente de determinação maior entre os modelos comparados.